In [ ]:
import matplotlib.pyplot as plt #kad visokias operacijas galima butu daryt
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

DirectMarketing = pd.read_csv("C:\\Users\\Samanta123\\Desktop\\projektas\\2019.csv")
